In [2]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import multi30k, Multi30k
from typing import Iterable, List

import torch
import torch.nn as nn
import math
import einops
from tokenizers import CharBPETokenizer




# We need to modify the URLs for the dataset since the links to the original dataset are broken
# Refer to https://github.com/pytorch/text/issues/1756#issuecomment-1163664163 for more info
multi30k.URL["train"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/training.tar.gz"
multi30k.URL["valid"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/validation.tar.gz"

SRC_LANGUAGE = 'de'
TGT_LANGUAGE = 'en'

/opt/anaconda3/lib/python3.11/site-packages/torchtext/data/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/opt/anaconda3/lib/python3.11/site-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/opt/anaconda3/lib/python3.11/site-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext i

In [3]:
# Import huggingface char-bpe tokenizer
en_tokenizer, de_tokenizer = CharBPETokenizer(), CharBPETokenizer()

special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']
en_tokenizer.add_special_tokens(special_symbols)
de_tokenizer.add_special_tokens(special_symbols)

# Train tokenizers
train_iter, test_iter, valid_iter = Multi30k(language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))

de_data, en_data = (list(zip(*train_iter)))
en_tokenizer.train_from_iterator(iterator=en_data)
de_tokenizer.train_from_iterator(iterator=de_data)

UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = tuple(en_tokenizer.encode(x).ids[0] for x in special_symbols)


/opt/anaconda3/lib/python3.11/site-packages/torchdata/datapipes/__init__.py:18: UserWarning: 
################################################################################
WARNING!
The 'datapipes', 'dataloader2' modules are deprecated and will be removed in a
future torchdata release! Please see https://github.com/pytorch/data/issues/1196
to learn more and leave feedback.
################################################################################

  deprecation_warning()


In [4]:
en_tokenizer.encode(en_data[0])#.tokens

Encoding(num_tokens=11, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [5]:
text_transform_en = lambda x : torch.tensor([BOS_IDX] + en_tokenizer.encode(x.rstrip("\n")).ids + [EOS_IDX])
text_transform_de = lambda x : torch.tensor([BOS_IDX] + de_tokenizer.encode(x.rstrip("\n")).ids + [EOS_IDX])



In [6]:
# torch.tensor(list(train_iter), dtype=torch.StringType)
# type(train_iter)
# (list(zip(*train_iter)))[0][9]
# en_data, de_data = (list(zip(*train_iter)))
# de_data_tokenized, en_data_tokenized = 
# map(lambda x : (x[0], x[1]), train_iter)#list(map(text_transform_de, de_data)), list(map(text_transform_de, de_data))
# # de_data_tokenized
# tokenized_multik3 = list(map(lambda x : (text_transform_de(x[0]), text_transform_en(x[1])), train_iter))

In [7]:
# len(tokenized_multik3), len(list(train_iter)), type(train_iter)
# torch.utils.data.datapipes.iter.sharding.ShardingFilterIterDataPipe(tokenized_multik3)

In [8]:
# list(torch.utils.data.datapipes.iter.sharding.ShardingFilterIterDataPipe(map(lambda x : (text_transform_de(x[0]), text_transform_en(x[1])), train_iter)))
tokenized_train_iter = torch.utils.data.datapipes.iter.sharding.ShardingFilterIterDataPipe(map(lambda x : (text_transform_de(x[0]), text_transform_en(x[1])), train_iter))
tokenized_test_iter = torch.utils.data.datapipes.iter.sharding.ShardingFilterIterDataPipe(map(lambda x : (text_transform_de(x[0]), text_transform_en(x[1])), test_iter))
tokenized_valid_iter = torch.utils.data.datapipes.iter.sharding.ShardingFilterIterDataPipe(map(lambda x : (text_transform_de(x[0]), text_transform_en(x[1])), valid_iter))

In [9]:
# len(list(tokenized_train_iter))

In [10]:
"""Position encoding"""
class PositionEncoding2(nn.Module):
    def __init__(self, d_embed):
        super(PositionEncoding2, self).__init__()
        self.d_embed = d_embed
        self.dropout = nn.Dropout(p=0.1)

    def get_position_encoding(self, seq_len):
        encoding = torch.zeros((seq_len, self.d_embed))
        dimensions = torch.arange(0, self.d_embed//2)
        timesteps = torch.arange(0, seq_len)

        encoding[:, 0::2] = torch.sin(torch.einsum('i,j -> ji', 1/(10000**(2*dimensions/self.d_embed)), timesteps))
        encoding[:, 1::2] = torch.cos(torch.einsum('i,j -> ji', 1/(10000**(2*dimensions/self.d_embed)), timesteps))
        
        return encoding
    
    def forward(self, inp):
        pos_encoding = nn.Parameter(self.get_position_encoding(seq_len=inp.shape[-2]), requires_grad=False)
        return self.dropout(inp + pos_encoding) # + dropout ?


In [11]:
"""Multihead attention"""
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, d_hidden, n_heads):
        super().__init__()#MultiheadAttention)
        self.d_hidden=d_hidden
        self.n_heads = n_heads
        self.W_q = nn.Parameter(
            nn.init.xavier_normal_(torch.zeros((n_heads, d_model, d_hidden//n_heads)))
        )
        self.W_k = nn.Parameter(
            nn.init.xavier_normal_(torch.zeros((n_heads, d_model, d_hidden//n_heads)))
        )
        self.W_v = nn.Parameter(
            nn.init.xavier_normal_(torch.zeros((n_heads, d_model, d_hidden//n_heads)))
        )

        self.W_o = nn.Linear(in_features=d_hidden, out_features=d_model, bias=False)

    def forward(self, q, k, v, get_attn_scores=False, mask=None, is_causal=False):
        mask_shape = (q.shape[-2], k.shape[-2]) 
        # print("mask shape: ", mask_shape)
        mask = mask if mask else (self.causal_mask(self.n_heads, mask_shape) if is_causal else self.empty_mask(self.n_heads, mask_shape))

        # Compute Q, K, V (matrix multiplication, batched along heads)
        q = einops.einsum(self.W_q, q, 'h d_model d_h, b T d_model -> b h  T d_h') # T is timesteps (sequence length)
        k = einops.einsum(self.W_k, k, 'h d_model d_h, b T d_model -> b h  T d_h')
        v = einops.einsum(self.W_v, v, 'h d_model d_h, b T d_model -> b h  T d_h')

        # print("q_shape: ", q.shape)
        # print("k_shape: ", k.shape)

        attn_scores = torch.softmax(
            einops.einsum(q, k, 'b h T_out d_h, b h T_in d_h -> b h T_out T_in')/math.sqrt(self.d_hidden//self.n_heads) + mask,#torch.FloatTensor(self.d_hidden//8)),
            dim=-1
        )

        attn_head_outs = einops.einsum(attn_scores, v, '... T_out T_in, ... T_in d_h -> ... T_out d_h')
        # print(attn_head_outs.shape)
        # print(attn_scores, "mask: ", mask)
        # print("mask: ", mask)
        concatted_outs = einops.rearrange(attn_head_outs, 'b h T_out d_h -> b T_out (h d_h)')
        concatted_outs = self.W_o(concatted_outs)

        return (concatted_outs, attn_scores) if get_attn_scores else concatted_outs
    
    @classmethod
    def causal_mask(cls, n_heads, mask_shape):
        return einops.repeat(
            torch.triu(
                torch.fill(torch.zeros(mask_shape),  -torch.inf),
                diagonal=1
                )
            , pattern='... -> k ...', k=n_heads
            )
    
    @classmethod
    def empty_mask(cls, n_heads, mask_shape):
        return einops.repeat(
            torch.zeros(mask_shape) , pattern='... -> k ...', k=n_heads
            )

In [46]:
"""Multihead attention"""
class MultiHeadAttention2(nn.Module):
    def __init__(self, d_model, d_hidden, n_heads):
        super().__init__()#MultiheadAttention)
        self.d_hidden=d_hidden
        self.n_heads = n_heads
        # self.W_q = nn.Parameter(
        #     nn.init.xavier_normal_(torch.zeros((n_heads, d_model, d_hidden)))
        # )
        # self.W_k = nn.Parameter(
        #     nn.init.xavier_normal_(torch.zeros((n_heads, d_model, d_hidden)))
        # )
        # self.W_v = nn.Parameter(
        #     nn.init.xavier_normal_(torch.zeros((n_heads, d_model, d_hidden)))
        # )

        self.W_q = nn.Linear(in_features=d_model, out_features=d_hidden)
        self.W_k = nn.Linear(in_features=d_model, out_features=d_hidden)
        self.W_v = nn.Linear(in_features=d_model, out_features=d_hidden)

        self.W_o = nn.Linear(in_features=d_hidden, out_features=d_model, bias=False)

    def forward(self, q, k, v, get_attn_scores=False, mask=None, is_causal=False):
        mask_shape = (q.shape[-2], k.shape[-2]) 
        # print("mask shape: ", mask_shape)
        mask = mask if mask else (self.causal_mask(self.n_heads, mask_shape) if is_causal else self.empty_mask(self.n_heads, mask_shape))
        # print("in q", k.shape)
        # Compute Q, K, V (matrix multiplication, batched along heads)
        q = self.W_q(q)#einops.einsum(self.W_q, q, 'h d_model d_h, b T d_model -> b h T d_h') # T is timesteps (sequence length)
        k = self.W_k(k)#einops.einsum(self.W_k, k, 'h d_model d_h, b T d_model -> b h T d_h')
        v = self.W_v(v)#einops.einsum(self.W_v, v, 'h d_model d_h, b T d_model -> b h T d_h')
        # print("out1 q", k.shape)
        # q = einops.rearrange(q, 'b T d_h -> b T h k', h=self.n_heads)
        q = einops.rearrange(q, 'b T (h w) -> b h T w', h=self.n_heads)
        k = einops.rearrange(k, 'b T (h w) -> b h T w', h=self.n_heads)
        v = einops.rearrange(v, 'b T (h w) -> b h T w', h=self.n_heads)
        # print("q_shape: ", q.shape)
        # print("k_shape: ", k.shape)
        # print("out2 q", k.shape)

        attn_scores = torch.softmax(
            einops.einsum(q, k, 'b h T_out d_h, b h T_in d_h -> b h T_out T_in')/math.sqrt(self.d_hidden//self.n_heads) + mask,#torch.FloatTensor(self.d_hidden//8)),
            dim=-1
        )

        attn_head_outs = einops.einsum(attn_scores, v, '... T_out T_in, ... T_in d_h -> ... T_out d_h')
        # print(attn_head_outs.shape)
        # print(attn_scores, "mask: ", mask)
        # print("mask: ", mask)
        concatted_outs = einops.rearrange(attn_head_outs, 'b h T_out d_h -> b T_out (h d_h)')
        concatted_outs = self.W_o(concatted_outs)

        return (concatted_outs, attn_scores) if get_attn_scores else concatted_outs
    
    @classmethod
    def causal_mask(cls, n_heads, mask_shape):
        return einops.repeat(
            torch.triu(
                torch.fill(torch.zeros(mask_shape),  -torch.inf),
                diagonal=1
                )
            , pattern='... -> k ...', k=n_heads
            )
    
    @classmethod
    def empty_mask(cls, n_heads, mask_shape):
        return einops.repeat(
            torch.zeros(mask_shape) , pattern='... -> k ...', k=n_heads
            )

In [47]:
b, h, T_out, d_h = 2, 3, 4, 6
q = torch.ones((b, h, T_out, d_h)) # target seq
k = torch.ones((b, h, T_out+2, d_h)) # inp seq

mask = MultiHeadAttention2.empty_mask(n_heads=h, mask_shape=(q.shape[-2], k.shape[-2]))#(k.shape[-2], q.shape[-2]))
# mask = 
# print(mask)
attn_scores = torch.softmax(
            einops.einsum(q, k, 'b h T_out d_h, b h T_in d_h -> b h T_out T_in')/math.sqrt(2) + mask,#torch.FloatTensor(self.d_hidden//8)),
            dim=-1
        )
attn_scores#.shape

tensor([[[[0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667],
          [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667],
          [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667],
          [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667]],

         [[0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667],
          [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667],
          [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667],
          [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667]],

         [[0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667],
          [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667],
          [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667],
          [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667]]],


        [[[0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667],
          [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667],
          [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667],
          [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667]],

         [[0.1667, 0.1667, 0.1

In [48]:
q.shape
q = torch.ones((b, T_out, d_h))
s = MultiHeadAttention2(d_model=d_h, d_hidden=d_h, n_heads=h)
ti = PositionEncoding2(d_embed=d_h)
q = ti(q)
q = s.W_q(q)
q.shape
print(q)
q = einops.rearrange(q, 'b T (h w) -> b h T w', h=h)#, w=d_h//h)
q.shape, q
print(q)

tensor([[[-1.0915, -0.5712,  1.2721, -0.3021,  0.7753,  0.3741],
         [-0.1946, -0.4819,  0.8439,  0.3889, -0.3510, -0.6231],
         [-0.8813, -0.0814,  1.3627,  0.1860,  0.1811, -0.1485],
         [-0.7180, -0.7588,  1.0270, -0.4516,  0.7069,  0.0185]],

        [[-0.6276, -0.3813,  0.9738, -0.0662,  0.3060,  0.1939],
         [-0.8500, -0.8558,  0.8711, -0.5391,  0.8880,  0.7814],
         [-1.5107,  0.5947,  1.3644,  0.5068, -0.0897, -0.0935],
         [-0.2300,  0.1176,  0.9877,  0.3936,  0.5128,  0.4988]]],
       grad_fn=<ViewBackward0>)
tensor([[[[-1.0915, -0.5712],
          [-0.1946, -0.4819],
          [-0.8813, -0.0814],
          [-0.7180, -0.7588]],

         [[ 1.2721, -0.3021],
          [ 0.8439,  0.3889],
          [ 1.3627,  0.1860],
          [ 1.0270, -0.4516]],

         [[ 0.7753,  0.3741],
          [-0.3510, -0.6231],
          [ 0.1811, -0.1485],
          [ 0.7069,  0.0185]]],


        [[[-0.6276, -0.3813],
          [-0.8500, -0.8558],
          [-1.51

In [49]:
class PosWiseFeedForward(nn.Module):
    def __init__(self, d_model=512, d_ff=2048):
        super(PosWiseFeedForward, self).__init__()
        
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        # FFN(x) = max(0, xW1 + b1)W2 + b2 ; f(x) = max(0, x) is relu
        return self.fc2(self.relu(self.fc1(x)))

In [207]:
# Encoder and Decoder
class EncoderLayer(nn.Module):
    def __init__(self, d_model, d_hidden, num_heads, d_ff, dropout=0.1, N_layers=6):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model=d_model, d_hidden=d_hidden, n_heads=num_heads)
        self.feed_forward = PosWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask=None, is_causal=False):
        #print("Q, K, V: ", x.size())
        attn_output = self.self_attn(x, x, x, mask=mask, is_causal=is_causal) # self attention
        x = self.norm1(x + self.dropout(attn_output)) # layer-norm + dropout + skip connection
        ff_output = self.feed_forward(x) # feed-forward
        x = self.norm2(x + self.dropout(ff_output)) # layer-norm + dropout + skip connection
        return x
    

    
class DecoderLayer(nn.Module):
    def __init__(self, d_model, d_hidden, num_heads, d_ff, dropout=0.1, N_layers=6):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, d_hidden, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, d_hidden, num_heads)
        self.feed_forward = PosWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, enc_output, src_mask, tgt_mask, is_causal=False):
        attn_output = self.self_attn(x, x, x, mask=tgt_mask, is_causal=is_causal) # self attention
        x = self.norm1(x + self.dropout(attn_output)) # layer-norm + dropout + skip connection
        attn_output = self.cross_attn(q=x, k=enc_output, v=enc_output, mask=src_mask)
        x = self.norm2(x + self.dropout(attn_output)) # layer-norm + droput + skip connection
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output)) # layer-norm + dropout + skip connection
        return x

In [208]:
class Transformer(nn.Module):
    def __init__(self, d_model, d_hidden, num_heads, d_ff,src_vocab_size, tgt_vocab_size, dropout=0, n_enc_layers=6, n_dec_layers=6):
        super(Transformer, self).__init__()
        # self.enc_layers = nn.Sequential(
        #     *(EncoderLayer(d_model=d_model,
        #                  d_hidden=d_hidden,
        #                  num_heads=num_heads,
        #                  d_ff=d_ff) for _ in range(n_enc_layers))
        # )
        # self.dec_layers = nn.Sequential(
        #     *(DecoderLayer(d_model=d_model,
        #                  d_hidden=d_hidden,
        #                  d_ff=d_ff,
        #                  num_heads=num_heads) for _ in range(n_enc_layers))
        # )

        self.enc_layers = nn.ModuleList(
            [EncoderLayer(d_model=d_model,
                         d_hidden=d_hidden,
                         num_heads=num_heads,
                         d_ff=d_ff) for _ in range(n_enc_layers)]
        )
        self.dec_layers = nn.ModuleList(
            [DecoderLayer(d_model=d_model,
                         d_hidden=d_hidden,
                         d_ff=d_ff,
                         num_heads=num_heads) for _ in range(n_dec_layers)]
        )

        self.pos_encoding_layer = PositionEncoding2(d_embed=d_model)

        self.src_embedding = nn.Embedding(embedding_dim=d_model, num_embeddings=src_vocab_size, padding_idx=PAD_IDX)
        self.tgt_embedding = nn.Embedding(embedding_dim=d_model, num_embeddings=tgt_vocab_size, padding_idx=PAD_IDX)

        self.final = nn.Linear(in_features=d_hidden, out_features=tgt_vocab_size)

    def forward(self, inp_seq, tgt_seq, inp_mask=None, tgt_mask=None, is_causal_tgt=False):
        

        inp_seq = self.src_embedding(inp_seq)
        inp_seq = self.pos_encoding_layer(inp_seq)

        for enc in self.enc_layers:
            inp_seq = enc(inp_seq, mask=inp_mask, is_causal=False)

        # inp_seq = self.enc_layers(inp_seq, mask=inp_mask, is_causal=False)

        tgt_seq = self.src_embedding(tgt_seq)
        # tgt_seq = self.dec_layers(x=tgt_seq, enc_out=inp_seq, mask=tgt_mask, is_causal=is_causal_tgt)
        for dec in self.dec_layers:
            tgt_seq = dec(x=tgt_seq, enc_output=inp_seq, src_mask=inp_mask, tgt_mask=tgt_mask, is_causal=is_causal_tgt)

        out_logits = self.final(tgt_seq)
        return out_logits





In [209]:
t = Transformer(d_model=8, d_hidden=8, num_heads=4, d_ff=10, src_vocab_size=3, tgt_vocab_size=7)
src, tgt = torch.arange(0, 3).tile(2, 1), torch.arange(0, 3).tile(2, 1)

src.shape, t(src, tgt)#.shape

(torch.Size([2, 3]),
 tensor([[[-0.5123,  0.6123,  0.4138, -0.6868, -0.0120, -0.6168,  0.0660],
          [-0.2988,  0.8658,  0.3645, -0.4842, -0.2050, -0.7479, -0.1059],
          [-0.4016,  0.6682,  0.4790, -0.5985,  0.2125, -0.4946, -0.1214]],
 
         [[-0.7274,  0.3957,  0.1740, -0.6914,  0.0929, -0.6376,  0.4161],
          [-0.7089,  0.5095,  0.2555, -0.6303,  0.3580, -0.3460,  0.2380],
          [-0.7822,  0.0296,  0.3872, -0.7767,  0.3205, -0.1270,  0.5384]]],
        grad_fn=<ViewBackward0>))

In [210]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    # print(batch)
    src_batch, tgt_batch = list(zip(*batch))
    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX).transpose(-1, -2)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX).transpose(-1, -2)

    return src_batch, tgt_batch 

p = list(zip(*tokenized_test_iter))
# list(tokenized_test_iter)

In [211]:
# de_tokenizer.decode((pad_sequence(p[0], padding_value=PAD_IDX).transpose(-1, -2))[1013].tolist())

In [212]:
# samp_batch = list(test_iter)#train_iter#list(train_iter)
# collated_samp = collate_fn(tokenized_test_iter)
# uncolled = list(tokenized_test_iter)


In [213]:
# collated_samp[0][len(collated_samp[0])-3]
# uncolled = list(tokenized_test_iter)
# l_iter = list(test_iter)

In [214]:
# len(uncolled), len(collated_samp[0]) #(1015, 42)

In [215]:
# uncolled[0], collated_samp[0][0], torch.max(collated_samp[0][len(collated_samp[0])-3])

In [216]:
# for t in train_dataloader:
#     print(t)
text_transform_en('one two three four'), text_transform_en('five six seven eight')

(tensor([   2,  313,  400,  826, 1276,    3]),
 tensor([   2, 1859, 2613, 5024, 3934,    3]))

In [217]:
from torch.utils.data import DataLoader


D_MODEL = 512
D_FF = 2048
N_HEADS = 8

BATCH_SIZE = 1#64
DEVICE = 'cpu'
LR = 0.00001#math.sqrt(D_MODEL/4000) # make this a schedule
BETAS = (0.9, 0.98)

# train_dataloader = DataLoader(tokenized_train_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)
# test_dataloader = DataLoader(tokenized_test_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)
# valid_dataloader = DataLoader(tokenized_valid_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)


def train_epoch(model, optimizer, data_loader=None):
    model.train()
    losses = 0
    # train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    # tokenized_train_iter = ('one two three four', 'five six seven eight')#torch.utils.data.datapipes.iter.sharding.ShardingFilterIterDataPipe(map(lambda x : (text_transform_de(x[0]), text_transform_en(x[1])), train_iter))
    # tokenized_train_iter = [(torch.Tensor([   2,  313,  400,  826, 1276,    3]).long(), torch.Tensor([   2, 1357, 1694,  210,  139, 1016, 5501, 8206, 1386,    3]).long())]
    # tokenized_train_iter = [(torch.Tensor([      2,  313,  400,  826, 1276,    3]).long(), torch.Tensor([   2, 1859, 2613, 5024, 3934,    3]).long())]
    tokenized_train_iter = torch.utils.data.datapipes.iter.sharding.ShardingFilterIterDataPipe(map(lambda x : (text_transform_de(x[0]), text_transform_en(x[1])), train_iter))

    train_dataloader = DataLoader(tokenized_train_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)

    loss_fn = nn.CrossEntropyLoss(ignore_index=PAD_IDX)#[PAD_IDX, BOS_IDX])

    data_count = 0
    import tqdm
    for src, tgt in tqdm.tqdm(train_dataloader):#data_loader):#train_dataloader):
        data_count+=1
        if data_count==100:
            break
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        out_logits = model(src, tgt[:, :-1], is_causal_tgt=True)

        # print("inputs: ", de_tokenizer.decode(src.squeeze().tolist()), en_tokenizer.decode(tgt[:, :-1]).squeeze().tolist())
        print("inputs: ", de_tokenizer.decode(list(src.squeeze()), skip_special_tokens=False), en_tokenizer.decode(list(tgt[:, :-1].squeeze()), skip_special_tokens=False))


        # print("out_logits shape: ", out_logits.shape)
        # print("tgt shape: ", tgt.shape) b seq vcb

        optimizer.zero_grad()

        # print("out_logits: ", out_logits.transpose(-1, -2), out_logits.transpose(-1, -2).shape)
        # print("tgt: ", tgt )
        

        loss = loss_fn(out_logits.transpose(-1, -2), tgt[:, 1:])

        loss.backward()
        optimizer.step()
        losses += loss.item()

    print("Trained on %s datapts"%data_count)

    return losses / len(list(train_dataloader))


def evaluate(model):
    model.eval()
    losses = 0
    loss_fn = nn.CrossEntropyLoss(ignore_index=PAD_IDX)
    val_iter = Multi30k(split='valid', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    val_dataloader = DataLoader(val_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)
    data_count=0
    for src, tgt in val_dataloader:
        data_count+=1
        if data_count==50:
            break
        src = src.to(DEVICE).T
        tgt = tgt.to(DEVICE).T

        tgt_input = tgt[:-1, :]

        print("src", src.T.shape)
        print("tgt", tgt.T.shape)

        # src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        # logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)
        output = model(src, tgt[:,:-1])

        

        loss = loss_fn(output.transpose(-1, -2), tgt)
        # loss = criterion(output.contiguous().view(-1, TGT_VOCAB_SIZE), tgt[:, 1:].contiguous().view(-1))
        losses += loss.item()

    return losses / len(list(val_dataloader))

In [218]:

model = Transformer(d_model=D_MODEL, 
                    d_hidden=D_MODEL, 
                    num_heads=N_HEADS, 
                    d_ff=D_FF, 
                    src_vocab_size=de_tokenizer.get_vocab_size(), 
                    tgt_vocab_size=en_tokenizer.get_vocab_size()
                    )

optimizer = torch.optim.Adam(betas=BETAS, eps=10e-9, lr=LR, params=model.parameters())


In [219]:
# de_tokenizer.get_vocab_size()
from timeit import default_timer as timer
NUM_EPOCHS = 1

for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    train_loss = train_epoch(model, optimizer)#, train_dataloader)
    end_time = timer()
    val_loss = 0#evaluate(model)
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))

0it [00:00, ?it/s]/opt/anaconda3/lib/python3.11/site-packages/torch/utils/data/datapipes/iter/combining.py:337: UserWarning: Some child DataPipes are not exhausted when __iter__ is called. We are resetting the buffer and each child DataPipe will read from the start again.
  warnings.warn("Some child DataPipes are not exhausted when __iter__ is called. We are resetting "


inputs:  <bos>Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche . <eos> <bos>Two young , White males are outside near many bushes .


1it [00:00,  1.25it/s]

inputs:  <bos>Mehrere Männer mit Schutzhelmen bedienen ein Antriebsradsystem . <eos> <bos>Several men in hard hats are operating a giant pulley system .


2it [00:01,  1.91it/s]

...

inputs:  <bos>Drei alte Männer sehen einem anderen Mann zu , wie er Fisch zubereitet . <eos> <bos>Three old men are watching another man prepare fish .
inputs:  <bos>Eine Frau einem einem weißen Pullunder mit einem grünen , wallenden Rock ein Lied singend auf der Bühne . <eos> <bos>A woman in a white tank top with a green flowing skirt , on stage singing a song .


97it [00:22,  5.11it/s]

inputs:  <bos>Zwei Männer und zwei Frauen , die auf Treppenstufen im Freien sitzen . <eos> <bos>Two men and two women sitting on steps outdoors .
inputs:  <bos>Ein brauner und ein schwarzer Labrador im Freien , wobei der schwarze Labrador ein Spielzeug in seinem Maul hat . <eos> <bos>A brown and black lab are outside and the black lab is catching a toy in its mouth .


99it [00:22,  4.36it/s]


inputs:  <bos>Junger männlicher Hockey - Goalie in roter Jacke duckt sich mit Stock beim Tor . <eos> <bos>Hockey goalie boy in red jacket crouches by goal , with stick .
Trained on 100 datapts
Epoch: 1, Train loss: 0.027, Val loss: 0.000, Epoch time = 25.822s


In [220]:
# torch.save(model, 'oct2+1')
# model = torch.load('oct2+1')

In [221]:
q, k, mask = torch.ones((32, 8, 23, 64)), torch.ones((32, 8, 24, 64)), torch.ones((23, 24))
attn_scores = torch.softmax(
            einops.einsum(q, k, 'b h T_out d_h, b h T_in d_h -> b h T_out T_in')/math.sqrt(2) + mask,#torch.FloatTensor(self.d_hidden//8)),
            dim=-1
        )

In [222]:
(einops.einsum(q, k, 'b h T_out d_h, b h T_in d_h -> b h T_out T_in')/math.sqrt(2)).shape# + mask

torch.Size([32, 8, 23, 24])

In [230]:
y_toks = [BOS_IDX] #+ list(text_transform_en('[BOS]'))
src_toks = de_tokenizer.encode(de_data[80]).ids
# src_toks = text_transform_en('<bos>')#torch.Tensor([   2,  313,  400,  826, 1276,    3]).long()
model = torch.load('it_kindof_works')
model.eval()
toks_generated=0
max_len=10#100
while y_toks[-1] != EOS_IDX and toks_generated < max_len:
    toks_generated+=1
    out_logits = model(torch.tensor(src_toks).unsqueeze(0), torch.tensor(y_toks).unsqueeze(0), is_causal_tgt=True)
    print(out_logits)
    # print(out_logits[:,-1].shape, "sfdsfhadsfhsdkjhfSDFfdf")
    idx_predicted = torch.argmax(out_logits[:,-1])
    y_toks.append(idx_predicted)

tensor([[[-1.1573, -2.0253, -1.4835,  ..., -0.8829, -0.1583, -1.5284]]],
       grad_fn=<ViewBackward0>)
tensor([[[-1.1573, -2.0253, -1.4835,  ..., -0.8829, -0.1583, -1.5284],
         [-1.1587, -1.4320, -1.3938,  ..., -0.6338, -0.1273, -1.6783]]],
       grad_fn=<ViewBackward0>)
tensor([[[-1.1573, -2.0253, -1.4835,  ..., -0.8829, -0.1583, -1.5284],
         [-1.1587, -1.4320, -1.3938,  ..., -0.6338, -0.1273, -1.6783],
         [-1.6576, -1.1608, -1.0758,  ..., -1.1079, -0.1805, -2.0747]]],
       grad_fn=<ViewBackward0>)
tensor([[[-1.1573, -2.0253, -1.4835,  ..., -0.8829, -0.1583, -1.5284],
         [-1.1587, -1.4320, -1.3938,  ..., -0.6338, -0.1273, -1.6783],
         [-1.6576, -1.1608, -1.0758,  ..., -1.1079, -0.1805, -2.0747],
         [-1.6403, -0.9788, -1.5941,  ..., -1.2258, -0.2059, -1.8604]]],
       grad_fn=<ViewBackward0>)
tensor([[[-1.1573, -2.0253, -1.4835,  ..., -0.8829, -0.1583, -1.5284],
         [-1.1587, -1.4320, -1.3938,  ..., -0.6338, -0.1273, -1.6783],
         [-1

In [231]:
(y_toks,
en_tokenizer.decode(y_toks),#[2456]),
en_data[80])

([2,
  tensor(120),
  tensor(167),
  tensor(162),
  tensor(101),
  tensor(96),
  tensor(3)],
 'A man in a .',
 'Two large tan dogs play along a sandy beach.')

In [229]:
# torch.save(model, 'it_doeskn_works')

In [226]:
# torch.zeros((2, 2, 3, 3)) + MultiHeadAttention.causal_mask(n_heads=2, mask_shape=(3,3))